## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-23-15-50-55 +0000,nypost,"AOC blows past financial disclosure deadline, ...",https://nypost.com/2025/08/23/us-news/aoc-blow...
1,2025-08-23-15-49-56 +0000,nyt,"After Gaza Famine Report, U.S. Is Mostly Silen...",https://www.nytimes.com/2025/08/23/world/middl...
2,2025-08-23-15-46-51 +0000,nypost,Macron says Trump’s ‘unpredictability’ key ass...,https://nypost.com/2025/08/23/world-news/macro...
3,2025-08-23-15-44-29 +0000,startribune,Who doesn’t love the Minnesota State Fair? Ple...,https://www.startribune.com/who-doesnt-love-th...
4,2025-08-23-15-40-27 +0000,nyt,"Faced With Trump’s English Mandate, Mexico’s T...",https://www.nytimes.com/2025/08/23/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2389/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,63
218,menendez,17
534,new,17
539,tariffs,12
219,parole,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
249,2025-08-22-18-08-00 +0000,wsj,The FBI searched the Maryland home and D.C. of...,https://www.wsj.com/politics/policy/fbi-raids-...,190
284,2025-08-22-16-19-00 +0000,wsj,Defense Secretary Pete Hegseth has authorized ...,https://www.wsj.com/politics/policy/defense-se...,134
39,2025-08-23-12-29-00 +0000,wsj,The Texas Senate approved a redistricting plan...,https://www.wsj.com/politics/policy/texas-cong...,122
27,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...,122
107,2025-08-23-03-00-00 +0000,wsj,Prospective ICE agents are going through a gru...,https://www.wsj.com/politics/policy/we-spent-s...,121


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
249,190,2025-08-22-18-08-00 +0000,wsj,The FBI searched the Maryland home and D.C. of...,https://www.wsj.com/politics/policy/fbi-raids-...
100,75,2025-08-23-03-46-41 +0000,cbc,Lyle Menendez denied parole in parents' 1989 m...,https://www.cbc.ca/news/world/lyle-menendez-de...
239,61,2025-08-22-18-33-00 +0000,wsj,Canada’s decision to drop tariffs on most U.S....,https://www.wsj.com/articles/canadas-rollback-...
27,59,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...
107,49,2025-08-23-03-00-00 +0000,wsj,Prospective ICE agents are going through a gru...,https://www.wsj.com/politics/policy/we-spent-s...
214,46,2025-08-22-19-27-10 +0000,nypost,Trump threatens to fire Fed’s Lisa Cook over a...,https://nypost.com/2025/08/22/business/trump-t...
284,45,2025-08-22-16-19-00 +0000,wsj,Defense Secretary Pete Hegseth has authorized ...,https://www.wsj.com/politics/policy/defense-se...
123,42,2025-08-23-01-00-00 +0000,wsj,Texas’ redistricting plan has set off a nation...,https://www.wsj.com/politics/a-partisan-shot-f...
33,36,2025-08-23-12-42-42 +0000,cbc,Kilmar Abrego Garcia freed from Tennessee jail...,https://www.cbc.ca/news/world/kilmar-abrego-ga...
134,32,2025-08-23-00-06-00 +0000,wsj,"Michael Faulkender, the second-highest officia...",https://www.wsj.com/politics/treasury-michael-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
